# 6c. Tree ensembles

In [ ]:
%matplotlib inline
import warnings;warnings.filterwarnings('ignore');import matplotlib.pyplot as plt;import numpy as np;import pandas as pd;import seaborn as sns;sns.set_context("notebook", font_scale=1.4);sns.set_style("whitegrid");import imp;compomics_import = imp.load_source('compomics_import', '../compomics_import.py');from IPython.core.display import HTML;css_file = '../my.css';HTML(open(css_file, "r").read())

In the previous chapter about bias and variance we have seen that averaging the predictions of several low bias high variance models each trained on different train set samples reduces the variance of the combined model (known as an **ensemble** or **meta-model**) while preserving a low bias.

A popular ensemble based learning algorithm that exploits this observation is the Random Forest learning algorithm. It computes an ensemble of complex decision trees (which are known to have low bias but high variance) by implementing what is called **boostrap aggregation (or bagging)**.   

The Random Forest learning algorithm trains $T$ decision trees $f_t(x,\theta)$ each on a random subset of the train set and then averages the predictions of each model according to:

$$\hat{f}(x,\theta) = \frac{1}{T} \sum_{t=1}^T f_t(x,\theta)$$

where $f_t$ is the $t^{th}$ tree. 

The Random Forest model consists essentially of bootstrap aggregated decision trees, where the bagging takes place at the level of the samples **and** the features. The learning algorithm for fitting a Random Forest model works as follows:

Let $n$ be the number of data points and $m$ the number of features in the train set. Construct $T$ decision trees by following these steps:

1. Sample $n$ data points at random - but **with replacement**, from the train set. These will be the training feature vectors for growing the tree. 

2. If there are $m$ features, a number $h<<m$ is specified such that at each node, $h$ features are selected at random out of the $m$ features and the best split on these $h$ is used to split the node. The value of $h$ is held constant during the forest growing.
 
3. Each tree is grown to the **largest extent possible**. There is no pruning or early stopping. 

Step 3 ensures that the bagged models are low bias by learning deep complex decision trees.

To illustrate the impact of averaging models trained on bagged train sets we consider the following two class classification task:

In [ ]:
data = pd.read_csv("dataset.csv")
y = data.pop("y")
X = data.values

plt.figure(figsize=(5,5))
for i, n, c in zip(range(2), "AB", "br"):
    idx = np.where(y == i)
    plt.scatter(X[idx, 0], X[idx, 1],
            c=c, cmap=plt.cm.Paired,
            s=20, edgecolor='k',
            label="Class %s" % n)
plt.legend(loc='upper right')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

We fit Random Forest meta-models to this data set for an increasing number of $T$ (usign the `RandomForestClassifier` function in scikit-learn):

In [ ]:
from sklearn.ensemble import RandomForestClassifier

plt.figure(figsize=(16, 5))
plt.subplot(1,4,1)
bdt = RandomForestClassifier(n_estimators=1)
bdt.fit(X, y)
compomics_import.plot_ensemble(bdt,X,y)   
plt.title("T=1")
plt.subplot(1,4,2)
bdt = RandomForestClassifier(n_estimators=3)
bdt.fit(X, y)
compomics_import.plot_ensemble(bdt,X,y)    
plt.title("T=3")
plt.subplot(1,4,3)
bdt = RandomForestClassifier(n_estimators=7)
bdt.fit(X, y)
compomics_import.plot_ensemble(bdt,X,y)    
plt.title("T=7")
plt.subplot(1,4,4)
bdt = RandomForestClassifier(n_estimators=50)
bdt.fit(X, y)
compomics_import.plot_ensemble(bdt,X,y)    
plt.title("T=50")
plt.show()

The meta-model classifies all test points in the blue region as class A (blue) and all test points in the brown region as class B (red). The meta-model with $T=50$ will show better generalization performance than with $T=1$. 

In the chapter about decision trees we estimated the expected generalization performance of an optimized decision tree on the TIS dataset. The following code estimates the generalization performance of the Random Forest meta-model:

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn import metrics

dataset = pd.read_csv("TIS.csv")
X = dataset.copy()
y = X.pop('label')

search_space = {'n_estimators':[10,20,50], 'max_features':[3,7,11,17]}
model = RandomForestClassifier(random_state=1,n_jobs=-1)
grid_search = GridSearchCV(model, param_grid=search_space, cv=5)

cv_predictions = cross_val_predict(grid_search, X, y, method="predict_proba")

fpr, tpr, thresholds = metrics.roc_curve(y, cv_predictions[:,1])
print metrics.auc(fpr, tpr)

You should observe better estimated performance.

A decision tree intrinsically computes feature importances by selecting appropriate split points. This information can be exploited to measure the importance of a feature in a decision tree: the more often a feature is used in the split points of a decision tree the more important that feature is. This notion of importance can be extended to decision tree ensembles by simply averaging the feature importance of each tree. In scikit-learn these feature importances are computed while fitting a model and are available through the `feature_importances_` property of a model.

The Random Forest learning algorithm reduces the variance of a low bias high variance decision tree model. We can also reduce the bias of a high bias low variance model using a technique called **boosting**. 

The idea is to transform an ensemble of low complex (high bias) models into a high complex (low bias) meta-model. In the context of boosting we talk about turning an ensemble of weak learners into a strong learner. 

A well known example is the **adaboost** learning algorithm. It computes a meta-model of the form:

$$\hat{f}(x,\theta) = \sum_{t=1}^T \alpha_t f_t(x,\theta),$$

Adaboost computes a strong classifier from an ensemble of weak classifiers as follows.

Let $n$ be the number of data points in the train set. Each data point $(x_i,y_i)$ with $x_i \in X$ and $y_i \in \{-1,1\}$ is assigned a weight $D(i)$. This weight is used by the weak classifier in the cost function to increase or decrease the contribution of a data point $x_i$ to the overal cost.

Initialize the weights $D_1(i) = 1/n$, $i = 1, 2, \dots, n$.

For $t = 1$ to $T$:
      
1. Fit a weak classifier $f_t(x,\theta)$ to the trainset data using weights $D_1(i)$.

2. Set $\alpha_t = \frac{1}{2} ln(\frac{1-error}{error})$.

3. Update weights:$$D_{t+1}(i)=\frac{D_t(i)exp({-\alpha_t y_i f_t(x_i,\theta)})}{Z_t},$$ where $Z_t$ is a normalizing factor that makes sure that $\sum{D_{t+1}(i)}=1$.

The $\alpha_t$ weights each model in the meta-model by the error it makes. Below the value of $\alpha$ is plotted against the error rate of a weak classifier:

In [ ]:
x = np.arange(start=0.01,stop=1,step=0.01)
y = 0.5 * np.log((1-x)/x)
plt.plot(x,y)
plt.xlabel("error")
plt.ylabel("alpha")
plt.show()

There are three bits of intuition to take from this graph:

- The weight $\alpha_t$ of a weak model $f_t(x_i,\theta)$ in the boosted meta-model increases exponentially as the error approaches 0. Better models are given exponentially more weight.

- The weight is zero if the error rate is 0.5. A model with 50% accuracy is no better than random guessing, so it is  ignored.

- The weight decreases exponentially as the error approaches 1. A negative weight is given to classifiers with worse than 50% accuracy. “Whatever that classifier says, do the opposite!”.

A typical weak classifier for adaboost is again a decision tree. However this time we want the decision tree learning algorithm to compute high bias low variance trees (we need weak learners). This is for instance achieved by severely limiting the depth of the decision trees that are constructed.

The following code shows how the decision boundary of a boosted decision tree with just one split improves as the number of these weak trees in the meta-model increases:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv("dataset.csv")
y = data.pop("y")
X = data.values

plt.figure(figsize=(16, 5))
plt.subplot(1,4,1)
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME",n_estimators=1)
bdt.fit(X, y)
compomics_import.plot_ensemble(bdt,X,y)    
plt.title("T=1")
plt.subplot(1,4,2)
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME",n_estimators=10)
bdt.fit(X, y)
compomics_import.plot_ensemble(bdt,X,y)    
plt.title("T=10")
plt.subplot(1,4,3)
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME",n_estimators=50)
bdt.fit(X, y)
compomics_import.plot_ensemble(bdt,X,y)    
plt.title("T=50")
plt.subplot(1,4,4)
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME",n_estimators=100)
bdt.fit(X, y)
compomics_import.plot_ensemble(bdt,X,y)    
plt.title("T=100")
plt.show()

In Adaboost the "shortcommings" of a weak learner are identified by high-weight data points. In a more recent approach to boosting, called **Gradient Boosting**, the "shortcommings" are identified by gradients.  

Gradient Boosting computes a strong classifier

$$\hat{f}(x,\theta) = \sum_{t=1}^T f_t(x,\theta),$$

from an ensemble of weak classifiers as follows.

1. Fit a model $f_1(x,\theta) = y$
2. Fit a model to the **residuals** $h_1(x) = y - f_1(x,\theta)$
3. Create a new model $f_2(x,\theta) = f_1(x,\theta)+h_1(x)$

We can then repeat steps 2 and 3 to compute a new model $f_3(x,\theta)$ that corrects the errors (residuals) of the previous model. This leads to a recursive procedure that can be make more mathematically consistent by introducing a model 

$$f_0(x,\theta) = \frac{1}{n}\sum_{i=1}^{n}{y_i}.$$

So $f_0$ predicts the average value of the label in the train set for each data point $x$.

Now we can define each subsequent $f_t$ recursively:

$$f_{t+1}(x,\theta) = f_{t}(x,\theta) + h_{t}(x),$$

where $h_{t}(x)$ is a (weak) learner.